<a href="https://colab.research.google.com/github/khushbooekhande/DrugRecommendationSystem/blob/master/Implementation/DiseasePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import LogisticRegression
import random

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_excel('/content/drive/My Drive/gtdt/Symptoms_dataset.xlsx')

In [5]:
data

,Disease,Count of Disease Occurrence,Symptom
0,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0008031_pain chest
1,NaN,NaN,UMLS:C0392680_shortness of breath
2,NaN,NaN,UMLS:C0012833_dizziness
3,NaN,NaN,UMLS:C0004093_asthenia
4,NaN,NaN,UMLS:C0085639_fall
...,...,...,...
1861,NaN,NaN,UMLS:C0425251_bedridden^UMLS:C0741453_bedridden
1862,NaN,NaN,UMLS:C0242453_prostatism
1863,UMLS:C0011127_decubitus ulcer,42.0,UMLS:C0232257_systolic murmur
1864,NaN,NaN,UMLS:C0871754_frail


In [0]:
data = data.fillna(method='ffill')
data.shape

(1866, 3)

In [0]:
# Clean Disease and Symptom Names
def clean_data(data):
    data_list = []
    data_name = data.replace('^','_').split('_')
    n = 1
    for names in data_name:
        if (n % 2 == 0):
            data_list.append(names)
        n += 1
    return data_list

In [0]:
disease_list = []
disease_symptom_dict = defaultdict(list)
disease_symptom_count = {}
count = 0

for idx, row in data.iterrows():
    
    # Get the Disease Names
    if (row['Disease'] !="\xc2\xa0") and (row['Disease'] != ""):
        disease = row['Disease']
        disease_list = clean_data(data=disease)
        count = row['Count of Disease Occurrence']

    # Get the Symptoms Corresponding to Diseases
    if (row['Symptom'] !="\xc2\xa0") and (row['Symptom'] != ""):
        symptom = row['Symptom']
        symptom_list = clean_data(data=symptom)
        for d in disease_list:
            for s in symptom_list:
                disease_symptom_dict[d].append(s)
            disease_symptom_count[d] = count

In [0]:
# Writing the data in a csv file

with open("/content/drive/My Drive/Symptoms_clean.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    for key,values in disease_symptom_dict.items():
        for v in values:
            key = str.encode(key).decode('utf-8')
            writer.writerow([key,v,disease_symptom_count[key]])

In [0]:
columns = ['Disease', 'Symptoms', 'Count']
data_clean = pd.read_csv("/content/drive/My Drive/gtdt/Symptoms_clean.csv", names = columns, encoding ="ISO-8859-1")

In [0]:
data_clean

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2125,affect labile,bedridden,45.0
2126,affect labile,prostatism,45.0
2127,decubitus ulcer,systolic murmur,42.0
2128,decubitus ulcer,frail,42.0


In [0]:
len(data_clean["Disease"].unique())

149

In [0]:
len(data_clean["Symptoms"].unique())

405

In [0]:
df = pd.DataFrame(data_clean)
df

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2125,affect labile,bedridden,45.0
2126,affect labile,prostatism,45.0
2127,decubitus ulcer,systolic murmur,42.0
2128,decubitus ulcer,frail,42.0


# **METHOD 1**

In [0]:
df1=df.drop('Count',axis=1)
df

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2125,affect labile,bedridden,45.0
2126,affect labile,prostatism,45.0
2127,decubitus ulcer,systolic murmur,42.0
2128,decubitus ulcer,frail,42.0


In [0]:
'''df_dc = df[['Disease', 'Count']]
df_dc'''

"df_dc = df[['Disease', 'Count']]\ndf_dc"

In [0]:
df_s = df1['Disease']
df_s = pd.DataFrame(df_s)
df_s

,Disease
0,hypertensive disease
1,hypertensive disease
2,hypertensive disease
3,hypertensive disease
4,hypertensive disease
...,...
2125,affect labile
2126,affect labile
2127,decubitus ulcer
2128,decubitus ulcer


In [0]:
df_1 = pd.get_dummies(df.Symptoms)
df_1

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_pivoted = pd.concat([df_s,df_1], axis=1)
df_pivoted

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,affect labile,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2126,affect labile,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2127,decubitus ulcer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2128,decubitus ulcer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_pivoted = df_pivoted.groupby('Disease').sum()
df_pivoted

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
Disease,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
HIV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PneumocystisÂ cariniiÂ pneumonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
accidentÂ cerebrovascular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
acquiredÂ immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tonic-clonic seizures,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
transient ischemic attack,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
tricuspid valve insufficiency,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
df_pivoted = df_pivoted.reset_index()

Disease prediction using the original dataset without considering the disease occurance count

In [0]:
cols = df_pivoted.columns
cols = cols[1:]
cols

Index(['Heberden's node', 'Murphy's sign', 'Stahli's line', 'abdomen acute',
       'abdominal bloating', 'abdominal tenderness', 'abnormal sensation',
       'abnormally hard consistency', 'abortion', 'abscess bacterial',
       ...
       'vision blurred', 'vomiting', 'weepiness', 'weight gain', 'welt',
       'wheelchair bound', 'wheezing', 'withdraw', 'worry', 'yellow sputum'],
      dtype='object', length=404)

In [0]:
X = df_pivoted[cols]
y = df_pivoted['Disease']

Multibinomial Naive bayes

In [0]:
mnb = MultinomialNB()
mnb = mnb.fit(X, y)
y_pred = mnb.predict(X)

In [0]:
pred_counts = 0
for i in range(0, len(y)):
  if y_pred[i]!= y[i]:
    pred_counts += 1
print("Total number of incorrect predictions:", pred_counts)
print("Accuracy of the MNB model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')

Total number of incorrect predictions: 15
Accuracy of the MNB model in Predicting Disease:  89.93288590604027 %


ExtraTree Classifier

In [0]:
ext = ExtraTreesClassifier(n_estimators=100, random_state=0)
ext = ext.fit(X, y)
y_pred = ext.predict(X)

In [0]:
pred_counts = 0
for i in range(0, len(y)):
  if y_pred[i]!= y[i]:
    pred_counts += 1
print("Total number of incorrect predictions:", pred_counts)
print("Accuracy of the ExtraTree model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')

Total number of incorrect predictions: 15
Accuracy of the ExtraTree model in Predicting Disease:  89.93288590604027 %


Decision tree classifier

In [0]:
for i in range(10, 111, 20):
  dtc = DecisionTreeClassifier(max_depth=i)
  dtc.fit(X,y)
  y_pred = dtc.predict(X)
  print("Accuracy of the ExtraTree classifier model in Predicting Disease: ", accuracy_score(y, y_pred)*100, '%')

Accuracy of the ExtraTree classifier model in Predicting Disease:  27.516778523489933 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  67.78523489932886 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  89.93288590604027 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  89.93288590604027 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  89.93288590604027 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  89.93288590604027 %


Support Vector Machine

In [0]:
svm = svm.SVC()
svm = svm.fit(X,y)
y_pred = svm.predict(X)
print("Accuracy of the SVM model in Predicting Disease: ", accuracy_score(y, y_pred)*100, '%')

Accuracy of the SVM model in Predicting Disease:  89.93288590604027 %


### Disease prediction using method 1

In [0]:
importances = dtc.feature_importances_
indices = np.argsort(importances)[::-1]

features = cols

In [0]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

Prediction using one symptom

In [0]:
feature_dict['shortness of breath']
symptomDR = [i/332 if i == 332 else i*0 for i in range(len(features))]
print(len(symptomDR))
symptomDR = np.array(symptomDR).reshape(1,len(symptomDR))

404


In [0]:
dtc.predict(symptomDR)

array(['decubitus ulcer'], dtype=object)

In [0]:
ext.predict(symptomDR)

array(['decubitus ulcer'], dtype=object)

In [0]:
mnb.predict(symptomDR)

array(['peripheral vascular disease'], dtype='<U36')

In [0]:
svm.predict(symptomDR)

array(['decubitus ulcer'], dtype=object)

Prediction using two symptoms

In [0]:
input_features = [feature_dict['pain chest'], feature_dict['shortness of breath']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [0]:
dtc.predict(arr)

array(['decubitus ulcer'], dtype=object)

In [0]:
ext.predict(arr)

array(['kidney disease'], dtype=object)

In [0]:
mnb.predict(arr)

array(['kidney disease'], dtype='<U36')

In [0]:
svm.predict(arr)

array(['kidney disease'], dtype=object)

Prediction using three features

In [0]:
input_features = [feature_dict['pain chest'], feature_dict['shortness of breath'],  feature_dict['asthenia']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [0]:
dtc.predict(arr)

array(['decubitus ulcer'], dtype=object)

In [0]:
ext.predict(arr)

array(['kidney disease'], dtype=object)

In [0]:
mnb.predict(symptomDR)

array(['peripheral vascular disease'], dtype='<U36')

In [0]:
svm.predict(symptomDR)

array(['decubitus ulcer'], dtype=object)

# **METHOD 2**

In [0]:
df_dc = df[['Disease', 'Count']]
df_dc

,Disease,Count
0,hypertensive disease,3363.0
1,hypertensive disease,3363.0
2,hypertensive disease,3363.0
3,hypertensive disease,3363.0
4,hypertensive disease,3363.0
...,...,...
2125,affect labile,45.0
2126,affect labile,45.0
2127,decubitus ulcer,42.0
2128,decubitus ulcer,42.0


In [0]:
df_dc.sort_values(by=['Disease'], inplace=True)
df_dc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Disease,Count
1442,Alzheimer's disease,101.0
1452,Alzheimer's disease,101.0
1459,Alzheimer's disease,101.0
1458,Alzheimer's disease,101.0
1457,Alzheimer's disease,101.0
...,...,...
1190,upper respiratory infection,135.0
1189,upper respiratory infection,135.0
1188,upper respiratory infection,135.0
1186,upper respiratory infection,135.0


In [0]:
df_dc=df_dc.drop_duplicates('Disease')
df_dc

,Disease,Count
1442,Alzheimer's disease,101.0
377,HIV,350.0
1361,PneumocystisÂ cariniiÂ pneumonia,113.0
119,accidentÂ cerebrovascular,885.0
361,acquiredÂ immuno-deficiency syndrome,350.0
...,...,...
1612,tonic-clonic seizures,92.0
842,transient ischemic attack,168.0
1432,tricuspid valve insufficiency,101.0
1103,ulcer peptic,143.0


In [0]:
df_pivoted = pd.concat([df_s,df_1], axis=1)
df_pivoted

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,affect labile,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2126,affect labile,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2127,decubitus ulcer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2128,decubitus ulcer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_pivoted = df_pivoted.groupby('Disease').sum()
df_pivoted

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
Disease,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
HIV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PneumocystisÂ cariniiÂ pneumonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
accidentÂ cerebrovascular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
acquiredÂ immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tonic-clonic seizures,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
transient ischemic attack,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
tricuspid valve insufficiency,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
data = df_pivoted.join(df_dc.set_index('Disease'), on='Disease')
data

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,Count
Disease,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,101.0
HIV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,350.0
PneumocystisÂ cariniiÂ pneumonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,113.0
accidentÂ cerebrovascular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,885.0
acquiredÂ immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tonic-clonic seizures,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92.0
transient ischemic attack,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,168.0
tricuspid valve insufficiency,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,101.0


In [0]:
data =data.reset_index()
data

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,Count
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,101.0
1,HIV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,350.0
2,PneumocystisÂ cariniiÂ pneumonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,113.0
3,accidentÂ cerebrovascular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,885.0
4,acquiredÂ immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,tonic-clonic seizures,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92.0
145,transient ischemic attack,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,168.0
146,tricuspid valve insufficiency,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,101.0
147,ulcer peptic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,143.0


Multiplying the rows

In [0]:
data['Count'] = data['Count'].astype(int)

In [0]:
data.dtypes

Disease            object
Heberden's node     uint8
Murphy's sign       uint8
Stahli's line       uint8
abdomen acute       uint8
                    ...  
wheezing            uint8
withdraw            uint8
worry               uint8
yellow sputum       uint8
Count               int64
Length: 406, dtype: object

In [0]:
df_new = pd.DataFrame([data.iloc[idx] 
                       for idx in data.index 
                       for i in range(data.iloc[idx]['Count'])]).reset_index(drop=True)

In [0]:
df_new.shape

(38839, 406)

In [0]:
df_new.head()

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,Count
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,101
1,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,101
2,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,101
3,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,101
4,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,101


In [0]:
cols = df_new.columns
cols = cols[1:-1]
cols

Index(['Heberden's node', 'Murphy's sign', 'Stahli's line', 'abdomen acute',
       'abdominal bloating', 'abdominal tenderness', 'abnormal sensation',
       'abnormally hard consistency', 'abortion', 'abscess bacterial',
       ...
       'vision blurred', 'vomiting', 'weepiness', 'weight gain', 'welt',
       'wheelchair bound', 'wheezing', 'withdraw', 'worry', 'yellow sputum'],
      dtype='object', length=404)

In [0]:
X = df_new[cols]
y = df_new['Disease']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [0]:
x_train.shape

(31071, 404)

Multibinomial Naive Bayes 

In [0]:
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)
y_pred = mnb.predict(x_test)
print("Accuracy of the MNB model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the MNB model in Predicting Disease:  86.84346035015447 %


Extratree Classifier

In [0]:
ext = ExtraTreesClassifier(n_estimators=100, random_state=0)
ext = ext.fit(x_train, y_train)
y_pred = ext.predict(x_test)
print("Accuracy of the ExtraTree classifier model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the ExtraTree classifier model in Predicting Disease:  86.84346035015447 %


Decision Tree Classifier

In [0]:
for i in range(10, 111, 20):
  dtc = DecisionTreeClassifier(max_depth=i)
  dtc.fit(x_train,y_train)
  y_pred = dtc.predict(x_test)
  print("Accuracy of the Decision classifier model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')
  

Accuracy of the Decision classifier model in Predicting Disease:  45.79042224510814 %
Accuracy of the Decision classifier model in Predicting Disease:  66.85118434603503 %
Accuracy of the Decision classifier model in Predicting Disease:  76.77651905252317 %
Accuracy of the Decision classifier model in Predicting Disease:  82.62100926879505 %
Accuracy of the Decision classifier model in Predicting Disease:  86.34140061791967 %
Accuracy of the Decision classifier model in Predicting Disease:  86.84346035015447 %


Support Vector Machine

In [0]:
svm = svm.SVC()
svm = svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)
print("Accuracy of the SVM model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the SVM model in Predicting Disease:  86.84346035015447 %


##Disease Prediction using method two

In [0]:
importances = dtc.feature_importances_
indices = np.argsort(importances)[::-1]

features = cols

In [0]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

Prediction using one symptom

In [0]:
feature_dict['shortness of breath']
symptomDR = [i/332 if i == 332 else i*0 for i in range(len(features))]
len(symptomDR)
symptomDR = np.array(symptomDR).reshape(1,len(symptomDR))

In [0]:
dtc.predict(symptomDR)

array(['delirium'], dtype=object)

In [0]:
ext.predict(symptomDR)

array(['decubitus ulcer'], dtype=object)

In [0]:
mnb.predict(symptomDR)

array(['peripheral vascular disease'], dtype='<U36')

Prediction using two symptoms

In [0]:
input_features = [feature_dict['pain chest'], feature_dict['shortness of breath']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [0]:
dtc.predict(arr)

array(['decubitus ulcer'], dtype=object)

In [0]:
ext.predict(arr)

array(['kidney disease'], dtype=object)

In [0]:
mnb.predict(arr)

array(['hypertensive disease'], dtype='<U36')

In [0]:
svm.predict(arr)

array(['kidney disease'], dtype=object)

Prediction using three symptoms

In [0]:
input_features = [feature_dict['pain chest'], feature_dict['shortness of breath'],  feature_dict['asthenia']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [0]:
dtc.predict(arr)

array(['decubitus ulcer'], dtype=object)

In [0]:
ext.predict(arr)

array(['kidney disease'], dtype=object)

In [0]:
mnb.predict(arr)

array(['hypertensive disease'], dtype='<U36')

In [0]:
svm.predict(arr)

array(['kidney disease'], dtype=object)

# **METHOD 3**

In [0]:
df

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2125,affect labile,bedridden,45.0
2126,affect labile,prostatism,45.0
2127,decubitus ulcer,systolic murmur,42.0
2128,decubitus ulcer,frail,42.0


In [0]:
df_dc = df[['Disease', 'Count']]
df_dc

,Disease,Count
0,hypertensive disease,3363.0
1,hypertensive disease,3363.0
2,hypertensive disease,3363.0
3,hypertensive disease,3363.0
4,hypertensive disease,3363.0
...,...,...
2125,affect labile,45.0
2126,affect labile,45.0
2127,decubitus ulcer,42.0
2128,decubitus ulcer,42.0


In [0]:
count = df['Count']
for i in count:
  df_dc = df_dc.replace(i, 200)
df_dc


,Disease,Count
0,hypertensive disease,200.0
1,hypertensive disease,200.0
2,hypertensive disease,200.0
3,hypertensive disease,200.0
4,hypertensive disease,200.0
...,...,...
2125,affect labile,200.0
2126,affect labile,200.0
2127,decubitus ulcer,200.0
2128,decubitus ulcer,200.0


In [0]:
df_s = df['Disease']
df_s = pd.DataFrame(df_s)
df_s

,Disease
0,hypertensive disease
1,hypertensive disease
2,hypertensive disease
3,hypertensive disease
4,hypertensive disease
...,...
2125,affect labile
2126,affect labile
2127,decubitus ulcer
2128,decubitus ulcer


In [0]:
df_new = pd.concat([df_s,df_1], axis=1)
df_new


,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,bleeding of vagina,bowel sounds decreased,bradycardia,bradykinesia,breakthrough pain,breath sounds decreased,breath-holding spell,breech presentation,bruit,burning sensation,cachexia,cardiomegaly,cardiovascular event,cardiovascular finding,catatonia,catching breath,charleyhorse,chest discomfort,chest tightness,chill,choke,cicatrisation,clammy skin,claudication,clonus,clumsiness,colic abdominal,consciousness clear,constipation,coordination abnormal,cough,cushingoid facies,cushingoidÂ habitus,cyanosis,cystic lesion,debilitation,decompensation,decreased body weight,decreased stool caliber,decreased translucency,diarrhea,difficulty,difficulty passing urine,disequilibrium,distended abdomen,distress respiratory,disturbed family,dizziness,dizzy spells,drool,drowsiness,dullness,dysarthria,dysdiadochokinesia,dysesthesia,dyspareunia,dyspnea,dyspnea on exertion,dysuria,ecchymosis,egophony,elation,emphysematous change,energy increased,enuresis,erythema,estrogen use,excruciating pain,exhaustion,extrapyramidal sign,extreme exhaustion,facial paresis,fall,fatigability,fatigue,fear of falling,fecaluria,feces in rectum,feeling hopeless,feeling strange,feeling suicidal,feels hot/feverish,fever,flare,flatulence,floppy,flushing,focal seizures,food intolerance,formication,frail,fremitus,frothy sputum,gag,gasping for breath,general discomfort,general unsteadiness,giddy mood,gravida 0,gravida 10,green sputum,groggy,guaiac positive,gurgle,hacking cough,haemoptysis,haemorrhage,hallucinations auditory,hallucinations visual,has religious belief,headache,heartburn,heavy feeling,heavy legs,hematochezia,hematocrit decreased,hematuria,heme positive,hemianopsia homonymous,hemiplegia,hemodynamically stable,hepatomegaly,hepatosplenomegaly,hirsutism,history of - blackout,hoard,hoarseness,homelessness,homicidal thoughts,hot flush,hunger,hydropneumothorax,hyperacusis,hypercapnia,hyperemesis,hyperhidrosis disorder,hyperkalemia,hypersomnia,hypersomnolence,hypertonicity,hyperventilation,hypesthesia,hypoalbuminemia,hypocalcemia result,hypokalemia,hypokinesia,hypometabolism,hyponatremia,hypoproteinemia,hypotension,"hypothermia, natural",hypotonic,hypoxemia,immobile,impaired cognition,inappropriate affect,incoherent,indifferent mood,intermenstrual heavy bleeding,intoxication,irritable mood,jugular venous distention,labored breathing,lameness,large-for-dates fetus,leftÂ atrialÂ hypertrophy,lesion,lethargy,lightheadedness,lip smacking,loose associations,low back pain,lung nodule,macerated skin,macule,malaise,mass in breast,mass of body structure,mediastinal shift,mental status changes,metastatic lesion,milky,moan,monoclonal,monocytosis,mood depressed,moody,motor retardation,muscle hypotonia,muscle twitch,myalgia,mydriasis,myoclonus,nasal discharge present,nasal flaring,nausea,nausea and vomiting,neck stiffness,neologism,nervousness,night sweat,nightmare,no known drug allergies,no status change,noisy respiration,non-productive cough,nonsmoker,numbness,numbness of hand,oliguria,orthopnea,orthostasis,out of breath,overweight,pain,pain abdominal,pain back,pain chest,pain foot,pain in lower limb,pain neck,painful swallowing,pallor,palpitation,panic,pansystolic murmur,para 1,para 2,paralyse,paraparesis,paresis,paresthesia,passed stones,patient non compliance,pericardial friction rub,phonophobia,photophobia,photopsia,pin-point pupils,pleuritic pain,pneumatouria,polydypsia,polymyalgia,polyuria,poor dentition,poor 

In [0]:
df_new = df_new.groupby('Disease').sum()
df_new

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,bleeding of vagina,bowel sounds decreased,bradycardia,bradykinesia,breakthrough pain,breath sounds decreased,breath-holding spell,breech presentation,bruit,burning sensation,cachexia,cardiomegaly,cardiovascular event,cardiovascular finding,catatonia,catching breath,charleyhorse,chest discomfort,chest tightness,chill,choke,cicatrisation,clammy skin,claudication,clonus,clumsiness,colic abdominal,consciousness clear,constipation,coordination abnormal,cough,cushingoid facies,cushingoidÂ habitus,cyanosis,cystic lesion,debilitation,decompensation,decreased body weight,decreased stool caliber,decreased translucency,diarrhea,difficulty,difficulty passing urine,disequilibrium,distended abdomen,distress respiratory,disturbed family,dizziness,dizzy spells,drool,drowsiness,dullness,dysarthria,dysdiadochokinesia,dysesthesia,dyspareunia,dyspnea,dyspnea on exertion,dysuria,ecchymosis,egophony,elation,emphysematous change,energy increased,enuresis,erythema,estrogen use,excruciating pain,exhaustion,extrapyramidal sign,extreme exhaustion,facial paresis,fall,fatigability,fatigue,fear of falling,fecaluria,feces in rectum,feeling hopeless,feeling strange,feeling suicidal,feels hot/feverish,fever,flare,flatulence,floppy,flushing,focal seizures,food intolerance,formication,frail,fremitus,frothy sputum,gag,gasping for breath,general discomfort,general unsteadiness,giddy mood,gravida 0,gravida 10,green sputum,groggy,guaiac positive,gurgle,hacking cough,haemoptysis,haemorrhage,hallucinations auditory,hallucinations visual,has religious belief,headache,heartburn,heavy feeling,heavy legs,hematochezia,hematocrit decreased,hematuria,heme positive,hemianopsia homonymous,hemiplegia,hemodynamically stable,hepatomegaly,hepatosplenomegaly,hirsutism,history of - blackout,hoard,hoarseness,homelessness,homicidal thoughts,hot flush,hunger,hydropneumothorax,hyperacusis,hypercapnia,hyperemesis,hyperhidrosis disorder,hyperkalemia,hypersomnia,hypersomnolence,hypertonicity,hyperventilation,hypesthesia,hypoalbuminemia,hypocalcemia result,hypokalemia,hypokinesia,hypometabolism,hyponatremia,hypoproteinemia,hypotension,"hypothermia, natural",hypotonic,hypoxemia,immobile,impaired cognition,inappropriate affect,incoherent,indifferent mood,intermenstrual heavy bleeding,intoxication,irritable mood,jugular venous distention,labored breathing,lameness,large-for-dates fetus,leftÂ atrialÂ hypertrophy,lesion,lethargy,lightheadedness,lip smacking,loose associations,low back pain,lung nodule,macerated skin,macule,malaise,mass in breast,mass of body structure,mediastinal shift,mental status changes,metastatic lesion,milky,moan,monoclonal,monocytosis,mood depressed,moody,motor retardation,muscle hypotonia,muscle twitch,myalgia,mydriasis,myoclonus,nasal discharge present,nasal flaring,nausea,nausea and vomiting,neck stiffness,neologism,nervousness,night sweat,nightmare,no known drug allergies,no status change,noisy respiration,non-productive cough,nonsmoker,numbness,numbness of hand,oliguria,orthopnea,orthostasis,out of breath,overweight,pain,pain abdominal,pain back,pain chest,pain foot,pain in lower limb,pain neck,painful swallowing,pallor,palpitation,panic,pansystolic murmur,para 1,para 2,paralyse,paraparesis,paresis,paresthesia,passed stones,patient non compliance,pericardial friction rub,phonophobia,photophobia,photopsia,pin-point pupils,pleuritic pain,pneumatouria,polydypsia,polymyalgia,polyuria,poor dentition,poor feeding,

In [0]:
df_dc=df_dc.drop_duplicates('Disease')
df_dc

,Disease,Count
0,hypertensive disease,200.0
13,diabetes,200.0
28,depression mental,200.0
49,depressive disorder,200.0
70,coronary arteriosclerosis,200.0
...,...,...
2069,ileus,200.0
2084,adhesion,200.0
2097,delusion,200.0
2118,affect labile,200.0


In [0]:
df_new = df_new.join(df_dc.set_index('Disease'), on='Disease')
df_new

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,bleeding of vagina,bowel sounds decreased,bradycardia,bradykinesia,breakthrough pain,breath sounds decreased,breath-holding spell,breech presentation,bruit,burning sensation,cachexia,cardiomegaly,cardiovascular event,cardiovascular finding,catatonia,catching breath,charleyhorse,chest discomfort,chest tightness,chill,choke,cicatrisation,clammy skin,claudication,clonus,clumsiness,colic abdominal,consciousness clear,constipation,coordination abnormal,cough,cushingoid facies,cushingoidÂ habitus,cyanosis,cystic lesion,debilitation,decompensation,decreased body weight,decreased stool caliber,decreased translucency,diarrhea,difficulty,difficulty passing urine,disequilibrium,distended abdomen,distress respiratory,disturbed family,dizziness,dizzy spells,drool,drowsiness,dullness,dysarthria,dysdiadochokinesia,dysesthesia,dyspareunia,dyspnea,dyspnea on exertion,dysuria,ecchymosis,egophony,elation,emphysematous change,energy increased,enuresis,erythema,estrogen use,excruciating pain,exhaustion,extrapyramidal sign,extreme exhaustion,facial paresis,fall,fatigability,fatigue,fear of falling,fecaluria,feces in rectum,feeling hopeless,feeling strange,feeling suicidal,feels hot/feverish,fever,flare,flatulence,floppy,flushing,focal seizures,food intolerance,formication,frail,fremitus,frothy sputum,gag,gasping for breath,general discomfort,general unsteadiness,giddy mood,gravida 0,gravida 10,green sputum,groggy,guaiac positive,gurgle,hacking cough,haemoptysis,haemorrhage,hallucinations auditory,hallucinations visual,has religious belief,headache,heartburn,heavy feeling,heavy legs,hematochezia,hematocrit decreased,hematuria,heme positive,hemianopsia homonymous,hemiplegia,hemodynamically stable,hepatomegaly,hepatosplenomegaly,hirsutism,history of - blackout,hoard,hoarseness,homelessness,homicidal thoughts,hot flush,hunger,hydropneumothorax,hyperacusis,hypercapnia,hyperemesis,hyperhidrosis disorder,hyperkalemia,hypersomnia,hypersomnolence,hypertonicity,hyperventilation,hypesthesia,hypoalbuminemia,hypocalcemia result,hypokalemia,hypokinesia,hypometabolism,hyponatremia,hypoproteinemia,hypotension,"hypothermia, natural",hypotonic,hypoxemia,immobile,impaired cognition,inappropriate affect,incoherent,indifferent mood,intermenstrual heavy bleeding,intoxication,irritable mood,jugular venous distention,labored breathing,lameness,large-for-dates fetus,leftÂ atrialÂ hypertrophy,lesion,lethargy,lightheadedness,lip smacking,loose associations,low back pain,lung nodule,macerated skin,macule,malaise,mass in breast,mass of body structure,mediastinal shift,mental status changes,metastatic lesion,milky,moan,monoclonal,monocytosis,mood depressed,moody,motor retardation,muscle hypotonia,muscle twitch,myalgia,mydriasis,myoclonus,nasal discharge present,nasal flaring,nausea,nausea and vomiting,neck stiffness,neologism,nervousness,night sweat,nightmare,no known drug allergies,no status change,noisy respiration,non-productive cough,nonsmoker,numbness,numbness of hand,oliguria,orthopnea,orthostasis,out of breath,overweight,pain,pain abdominal,pain back,pain chest,pain foot,pain in lower limb,pain neck,painful swallowing,pallor,palpitation,panic,pansystolic murmur,para 1,para 2,paralyse,paraparesis,paresis,paresthesia,passed stones,patient non compliance,pericardial friction rub,phonophobia,photophobia,photopsia,pin-point pupils,pleuritic pain,pneumatouria,polydypsia,polymyalgia,polyuria,poor dentition,poor feeding,

In [0]:
df_new = df_new.reset_index()

In [0]:
df_new.drop(df_new.index[:], inplace=True)
df_new

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,bleeding of vagina,bowel sounds decreased,bradycardia,bradykinesia,breakthrough pain,breath sounds decreased,breath-holding spell,breech presentation,bruit,burning sensation,cachexia,cardiomegaly,cardiovascular event,cardiovascular finding,catatonia,catching breath,charleyhorse,chest discomfort,chest tightness,chill,choke,cicatrisation,clammy skin,claudication,clonus,clumsiness,colic abdominal,consciousness clear,constipation,coordination abnormal,cough,cushingoid facies,cushingoidÂ habitus,cyanosis,cystic lesion,debilitation,decompensation,decreased body weight,decreased stool caliber,decreased translucency,diarrhea,difficulty,difficulty passing urine,disequilibrium,distended abdomen,distress respiratory,disturbed family,dizziness,dizzy spells,drool,drowsiness,dullness,dysarthria,dysdiadochokinesia,dysesthesia,dyspareunia,dyspnea,dyspnea on exertion,dysuria,ecchymosis,egophony,elation,emphysematous change,energy increased,enuresis,erythema,estrogen use,excruciating pain,exhaustion,extrapyramidal sign,extreme exhaustion,facial paresis,fall,fatigability,fatigue,fear of falling,fecaluria,feces in rectum,feeling hopeless,feeling strange,feeling suicidal,feels hot/feverish,fever,flare,flatulence,floppy,flushing,focal seizures,food intolerance,formication,frail,fremitus,frothy sputum,gag,gasping for breath,general discomfort,general unsteadiness,giddy mood,gravida 0,gravida 10,green sputum,groggy,guaiac positive,gurgle,hacking cough,haemoptysis,haemorrhage,hallucinations auditory,hallucinations visual,has religious belief,headache,heartburn,heavy feeling,heavy legs,hematochezia,hematocrit decreased,hematuria,heme positive,hemianopsia homonymous,hemiplegia,hemodynamically stable,hepatomegaly,hepatosplenomegaly,hirsutism,history of - blackout,hoard,hoarseness,homelessness,homicidal thoughts,hot flush,hunger,hydropneumothorax,hyperacusis,hypercapnia,hyperemesis,hyperhidrosis disorder,hyperkalemia,hypersomnia,hypersomnolence,hypertonicity,hyperventilation,hypesthesia,hypoalbuminemia,hypocalcemia result,hypokalemia,hypokinesia,hypometabolism,hyponatremia,hypoproteinemia,hypotension,"hypothermia, natural",hypotonic,hypoxemia,immobile,impaired cognition,inappropriate affect,incoherent,indifferent mood,intermenstrual heavy bleeding,intoxication,irritable mood,jugular venous distention,labored breathing,lameness,large-for-dates fetus,leftÂ atrialÂ hypertrophy,lesion,lethargy,lightheadedness,lip smacking,loose associations,low back pain,lung nodule,macerated skin,macule,malaise,mass in breast,mass of body structure,mediastinal shift,mental status changes,metastatic lesion,milky,moan,monoclonal,monocytosis,mood depressed,moody,motor retardation,muscle hypotonia,muscle twitch,myalgia,mydriasis,myoclonus,nasal discharge present,nasal flaring,nausea,nausea and vomiting,neck stiffness,neologism,nervousness,night sweat,nightmare,no known drug allergies,no status change,noisy respiration,non-productive cough,nonsmoker,numbness,numbness of hand,oliguria,orthopnea,orthostasis,out of breath,overweight,pain,pain abdominal,pain back,pain chest,pain foot,pain in lower limb,pain neck,painful swallowing,pallor,palpitation,panic,pansystolic murmur,para 1,para 2,paralyse,paraparesis,paresis,paresthesia,passed stones,patient non compliance,pericardial friction rub,phonophobia,photophobia,photopsia,pin-point pupils,pleuritic pain,pneumatouria,polydypsia,polymyalgia,polyuria,poor dentition,poor 

In [0]:
dict1 = defaultdict(list)
for i in df['Disease'].unique():
  val = df.loc[df['Disease'] == i]
  symp = val['Symptoms']
  for s in symp:
    dict1[i].append(s)
print(dict1)

defaultdict(<class 'list'>, {'hypertensive disease': ['pain chest', 'shortness of breath', 'dizziness', 'asthenia', 'fall', 'syncope', 'vertigo', 'sweat', 'sweating increased', 'palpitation', 'nausea', 'angina pectoris', 'pressure chest'], 'diabetes': ['polyuria', 'polydypsia', 'shortness of breath', 'pain chest', 'asthenia', 'nausea', 'orthopnea', 'rale', 'sweat', 'sweating increased', 'unresponsiveness', 'mental status changes', 'vertigo', 'vomiting', 'labored breathing'], 'depression mental': ['feeling suicidal', 'suicidal', 'hallucinations auditory', 'feeling hopeless', 'weepiness', 'sleeplessness', 'motor retardation', 'irritable mood', 'blackout', 'mood depressed', 'hallucinations visual', 'worry', 'agitation', 'tremor', 'intoxication', 'verbal auditory hallucinations', 'energy increased', 'difficulty', 'nightmare', 'unable to concentrate', 'homelessness'], 'depressive disorder': ['feeling suicidal', 'suicidal', 'hallucinations auditory', 'feeling hopeless', 'weepiness', 'sleeple

In [0]:
symptoms = df_new.columns
symptoms_list = symptoms[1:-1]
symptoms_list

Index(['Heberden's node', 'Murphy's sign', 'Stahli's line', 'abdomen acute',
       'abdominal bloating', 'abdominal tenderness', 'abnormal sensation',
       'abnormally hard consistency', 'abortion', 'abscess bacterial',
       ...
       'vision blurred', 'vomiting', 'weepiness', 'weight gain', 'welt',
       'wheelchair bound', 'wheezing', 'withdraw', 'worry', 'yellow sputum'],
      dtype='object', length=404)

In [0]:
for disease, actual_symptoms in dict1.items():
  dict_value_rand = {}
  for i,symptom in enumerate(actual_symptoms):
    #print(symptom)
    random_numbers = random.sample(range(100), int(i)) 
    if i == 0:
      random_numbers=[-1]
  #print(type(random_numbers))
    dict_value_rand[symptom] = random_numbers
  print(dict_value_rand)
  for i in range(100):
    count = 0
    new_row = {}
    new_row['Disease'] = disease
    for s in symptoms_list:
      #print(s)
      if s in actual_symptoms and i not in dict_value_rand.get(s):
        #print("i am in here")
        value = 1
        count += 1
      else:
        value = 0
      new_row[s]=value
    print(new_row)
    df_new = df_new.append(new_row, ignore_index=True)
print(count)


In [0]:
df_new

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,bleeding of vagina,bowel sounds decreased,bradycardia,bradykinesia,breakthrough pain,breath sounds decreased,breath-holding spell,breech presentation,bruit,burning sensation,cachexia,cardiomegaly,cardiovascular event,cardiovascular finding,catatonia,catching breath,charleyhorse,chest discomfort,chest tightness,chill,choke,cicatrisation,clammy skin,claudication,clonus,clumsiness,colic abdominal,consciousness clear,constipation,coordination abnormal,cough,cushingoid facies,cushingoidÂ habitus,cyanosis,cystic lesion,debilitation,decompensation,decreased body weight,decreased stool caliber,decreased translucency,diarrhea,difficulty,difficulty passing urine,disequilibrium,distended abdomen,distress respiratory,disturbed family,dizziness,dizzy spells,drool,drowsiness,dullness,dysarthria,dysdiadochokinesia,dysesthesia,dyspareunia,dyspnea,dyspnea on exertion,dysuria,ecchymosis,egophony,elation,emphysematous change,energy increased,enuresis,erythema,estrogen use,excruciating pain,exhaustion,extrapyramidal sign,extreme exhaustion,facial paresis,fall,fatigability,fatigue,fear of falling,fecaluria,feces in rectum,feeling hopeless,feeling strange,feeling suicidal,feels hot/feverish,fever,flare,flatulence,floppy,flushing,focal seizures,food intolerance,formication,frail,fremitus,frothy sputum,gag,gasping for breath,general discomfort,general unsteadiness,giddy mood,gravida 0,gravida 10,green sputum,groggy,guaiac positive,gurgle,hacking cough,haemoptysis,haemorrhage,hallucinations auditory,hallucinations visual,has religious belief,headache,heartburn,heavy feeling,heavy legs,hematochezia,hematocrit decreased,hematuria,heme positive,hemianopsia homonymous,hemiplegia,hemodynamically stable,hepatomegaly,hepatosplenomegaly,hirsutism,history of - blackout,hoard,hoarseness,homelessness,homicidal thoughts,hot flush,hunger,hydropneumothorax,hyperacusis,hypercapnia,hyperemesis,hyperhidrosis disorder,hyperkalemia,hypersomnia,hypersomnolence,hypertonicity,hyperventilation,hypesthesia,hypoalbuminemia,hypocalcemia result,hypokalemia,hypokinesia,hypometabolism,hyponatremia,hypoproteinemia,hypotension,"hypothermia, natural",hypotonic,hypoxemia,immobile,impaired cognition,inappropriate affect,incoherent,indifferent mood,intermenstrual heavy bleeding,intoxication,irritable mood,jugular venous distention,labored breathing,lameness,large-for-dates fetus,leftÂ atrialÂ hypertrophy,lesion,lethargy,lightheadedness,lip smacking,loose associations,low back pain,lung nodule,macerated skin,macule,malaise,mass in breast,mass of body structure,mediastinal shift,mental status changes,metastatic lesion,milky,moan,monoclonal,monocytosis,mood depressed,moody,motor retardation,muscle hypotonia,muscle twitch,myalgia,mydriasis,myoclonus,nasal discharge present,nasal flaring,nausea,nausea and vomiting,neck stiffness,neologism,nervousness,night sweat,nightmare,no known drug allergies,no status change,noisy respiration,non-productive cough,nonsmoker,numbness,numbness of hand,oliguria,orthopnea,orthostasis,out of breath,overweight,pain,pain abdominal,pain back,pain chest,pain foot,pain in lower limb,pain neck,painful swallowing,pallor,palpitation,panic,pansystolic murmur,para 1,para 2,paralyse,paraparesis,paresis,paresthesia,passed stones,patient non compliance,pericardial friction rub,phonophobia,photophobia,photopsia,pin-point pupils,pleuritic pain,pneumatouria,polydypsia,polymyalgia,polyuria,poor dentition,poor 

In [0]:
pd.set_option('display.max_columns',None)

In [0]:
val = df_new.loc[df_new['Disease'] == 'hypertensive disease']

In [0]:
val

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,bleeding of vagina,bowel sounds decreased,bradycardia,bradykinesia,breakthrough pain,breath sounds decreased,breath-holding spell,breech presentation,bruit,burning sensation,cachexia,cardiomegaly,cardiovascular event,cardiovascular finding,catatonia,catching breath,charleyhorse,chest discomfort,chest tightness,chill,choke,cicatrisation,clammy skin,claudication,clonus,clumsiness,colic abdominal,consciousness clear,constipation,coordination abnormal,cough,cushingoid facies,cushingoidÂ habitus,cyanosis,cystic lesion,debilitation,decompensation,decreased body weight,decreased stool caliber,decreased translucency,diarrhea,difficulty,difficulty passing urine,disequilibrium,distended abdomen,distress respiratory,disturbed family,dizziness,dizzy spells,drool,drowsiness,dullness,dysarthria,dysdiadochokinesia,dysesthesia,dyspareunia,dyspnea,dyspnea on exertion,dysuria,ecchymosis,egophony,elation,emphysematous change,energy increased,enuresis,erythema,estrogen use,excruciating pain,exhaustion,extrapyramidal sign,extreme exhaustion,facial paresis,fall,fatigability,fatigue,fear of falling,fecaluria,feces in rectum,feeling hopeless,feeling strange,feeling suicidal,feels hot/feverish,fever,flare,flatulence,floppy,flushing,focal seizures,food intolerance,formication,frail,fremitus,frothy sputum,gag,gasping for breath,general discomfort,general unsteadiness,giddy mood,gravida 0,gravida 10,green sputum,groggy,guaiac positive,gurgle,hacking cough,haemoptysis,haemorrhage,hallucinations auditory,hallucinations visual,has religious belief,headache,heartburn,heavy feeling,heavy legs,hematochezia,hematocrit decreased,hematuria,heme positive,hemianopsia homonymous,hemiplegia,hemodynamically stable,hepatomegaly,hepatosplenomegaly,hirsutism,history of - blackout,hoard,hoarseness,homelessness,homicidal thoughts,hot flush,hunger,hydropneumothorax,hyperacusis,hypercapnia,hyperemesis,hyperhidrosis disorder,hyperkalemia,hypersomnia,hypersomnolence,hypertonicity,hyperventilation,hypesthesia,hypoalbuminemia,hypocalcemia result,hypokalemia,hypokinesia,hypometabolism,hyponatremia,hypoproteinemia,hypotension,"hypothermia, natural",hypotonic,hypoxemia,immobile,impaired cognition,inappropriate affect,incoherent,indifferent mood,intermenstrual heavy bleeding,intoxication,irritable mood,jugular venous distention,labored breathing,lameness,large-for-dates fetus,leftÂ atrialÂ hypertrophy,lesion,lethargy,lightheadedness,lip smacking,loose associations,low back pain,lung nodule,macerated skin,macule,malaise,mass in breast,mass of body structure,mediastinal shift,mental status changes,metastatic lesion,milky,moan,monoclonal,monocytosis,mood depressed,moody,motor retardation,muscle hypotonia,muscle twitch,myalgia,mydriasis,myoclonus,nasal discharge present,nasal flaring,nausea,nausea and vomiting,neck stiffness,neologism,nervousness,night sweat,nightmare,no known drug allergies,no status change,noisy respiration,non-productive cough,nonsmoker,numbness,numbness of hand,oliguria,orthopnea,orthostasis,out of breath,overweight,pain,pain abdominal,pain back,pain chest,pain foot,pain in lower limb,pain neck,painful swallowing,pallor,palpitation,panic,pansystolic murmur,para 1,para 2,paralyse,paraparesis,paresis,paresthesia,passed stones,patient non compliance,pericardial friction rub,phonophobia,photophobia,photopsia,pin-point pupils,pleuritic pain,pneumatouria,polydypsia,polymyalgia,polyuria,poor dentition,poor 

In [0]:
df_new = df_new.replace([np.inf, -np.inf], np.nan)

In [0]:
df_new=df_new.fillna(0)

In [0]:
cols = df_new.columns
cols = cols[1:-1]
cols.shape

(404,)

In [0]:
X = df_new[cols]
y = df_new['Disease']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

Multinomial Naive Bayes

In [0]:
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)
y_pred = mnb.predict(x_test)
print("Accuracy of the MNB model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')


Accuracy of the MNB model in Predicting Disease:  86.90064260998517 %


In [0]:
ext = ExtraTreesClassifier(n_estimators=100, random_state=0)
ext = ext.fit(x_train, y_train)
y_pred = ext.predict(x_test)
print("Accuracy of the ExtraTree classifier model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the ExtraTree classifier model in Predicting Disease:  88.18586258032624 %


In [0]:
for i in range(10, 121, 20):
  dtc = DecisionTreeClassifier(max_depth=i)
  dtc = dtc.fit(x_train,y_train)
  y_pred = dtc.predict(x_test)
  print("Accuracy of the DecisionTree classifier model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the DecisionTree classifier model in Predicting Disease:  11.61641127039051 %
Accuracy of the DecisionTree classifier model in Predicting Disease:  50.81562036579338 %
Accuracy of the DecisionTree classifier model in Predicting Disease:  65.52150271873455 %
Accuracy of the DecisionTree classifier model in Predicting Disease:  76.34700939199209 %
Accuracy of the DecisionTree classifier model in Predicting Disease:  85.39298072170044 %
Accuracy of the DecisionTree classifier model in Predicting Disease:  86.95007414730598 %


In [0]:
svm = svm.SVC()
svm = svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)
print("Accuracy of the SVM model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the SVM model in Predicting Disease:  86.99950568462678 %


Disease prediction using method 3

In [0]:
importances = dtc.feature_importances_
indices = np.argsort(importances)[::-1]

features = cols

In [0]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

Prediction using one symptom

In [0]:
feature_dict['pain chest']
symptomDR = [i/332 if i == 332 else i*0 for i in range(len(features))]
len(symptomDR)
symptomDR = np.array(symptomDR).reshape(1,len(symptomDR))

In [0]:
dtc.predict(symptomDR)

array(['overload fluid'], dtype=object)

In [0]:
ext.predict(symptomDR)

array(['decubitus ulcer'], dtype=object)

In [0]:
mnb.predict(symptomDR)

array(['peripheral vascular disease'], dtype='<U36')

In [0]:
svm.predict(symptomDR)

array(['decubitus ulcer'], dtype=object)

Prediction using two symptoms

In [0]:
input_features = [feature_dict['pain chest'], feature_dict['shortness of breath']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [0]:
dtc.predict(arr)

array(['pyelonephritis'], dtype=object)

In [0]:
ext.predict(arr)

array(['kidney disease'], dtype=object)

In [0]:
mnb.predict(arr)

array(['coronary heart disease'], dtype='<U36')

In [0]:
svm.predict(arr)

array(['kidney disease'], dtype=object)

Prediction using three symptoms

In [0]:
input_features = [feature_dict['pain chest'], feature_dict['asthenia'],  feature_dict['shortness of breath']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [0]:
dtc.predict(arr)

array(['pyelonephritis'], dtype=object)

In [0]:
ext.predict(arr)

array(['kidney disease'], dtype=object)

In [0]:
mnb.predict(arr)

array(['hypertensive disease'], dtype='<U36')

In [0]:
svm.predict(arr)

array(['kidney disease'], dtype=object)

Prediction using four symptoms

In [0]:
input_features = [feature_dict['pain chest'], feature_dict['asthenia'],  feature_dict['shortness of breath'],  feature_dict['syncope']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  elif i == input_features[3]:
    i = int(i/input_features[3])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [0]:
dtc.predict(arr)

array(['pyelonephritis'], dtype=object)

In [0]:
ext.predict(arr)

array(['hypertensive disease'], dtype=object)

In [0]:
mnb.predict(arr)

array(['hypertensive disease'], dtype='<U36')

In [0]:
svm.predict(arr)

array(['hypertensive disease'], dtype=object)

# **Method 4**

In [7]:
data_clean.head()

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0


In [0]:
df = pd.DataFrame(data_clean)

In [0]:
df_s = data_clean['Disease']

In [0]:
df_1 = pd.get_dummies(df.Symptoms)

In [0]:
df_pivoted = pd.concat([df_s,df_1], axis=1)

In [0]:
df_pivoted.drop_duplicates(keep='first',inplace=True)

In [16]:
df_pivoted[:5]

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
cols = df_pivoted.columns

In [0]:
cols = cols[1:]

In [19]:
print(df_pivoted['Disease'].value_counts().unique())

[28 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  3]


In [0]:
df_pivoted_1 = df_pivoted.groupby('Disease').head(3)
df_pivoted_2 = df_pivoted.groupby('Disease').head(5)
df_pivoted_3 = df_pivoted.groupby('Disease').head(7)
df_pivoted_4 = df_pivoted.groupby('Disease').head(10)
df_pivoted_5 = df_pivoted.groupby('Disease').head(12)
df_pivoted_6 = df_pivoted.groupby('Disease').head(15)
df_pivoted_7 = df_pivoted.groupby('Disease').head(17)
df_pivoted_8 = df_pivoted.groupby('Disease').head(20)
df_pivoted_9 = df_pivoted.groupby('Disease').head(24)
df_pivoted_10 = df_pivoted.groupby('Disease').head(28)

In [0]:
df_pivoted_1 = df_pivoted_1.groupby('Disease').sum().reset_index()
df_pivoted_2 = df_pivoted_2.groupby('Disease').sum().reset_index()
df_pivoted_3 = df_pivoted_3.groupby('Disease').sum().reset_index()
df_pivoted_4 = df_pivoted_4.groupby('Disease').sum().reset_index()
df_pivoted_5 = df_pivoted_5.groupby('Disease').sum().reset_index()
df_pivoted_6 = df_pivoted_6.groupby('Disease').sum().reset_index()
df_pivoted_7 = df_pivoted_7.groupby('Disease').sum().reset_index()
df_pivoted_8 = df_pivoted_8.groupby('Disease').sum().reset_index()
df_pivoted_9 = df_pivoted_9.groupby('Disease').sum().reset_index()
df_pivoted_10 = df_pivoted_10.groupby('Disease').sum().reset_index()

In [0]:
symptom_data = pd.concat([df_pivoted_1,df_pivoted_2,df_pivoted_3,df_pivoted_4,df_pivoted_5,
                      df_pivoted_6,df_pivoted_7,df_pivoted_8, df_pivoted_9, df_pivoted_10], axis=0, sort=True)

In [29]:
print(symptom_data['Disease'].value_counts())

malignant neoplasm of lung    10
carcinoma of lung             10
endocarditis                  10
delusion                      10
Alzheimer's disease           10
                              ..
encephalopathy                10
hyperbilirubinemia            10
gastroenteritis               10
tonic-clonic epilepsy         10
ulcer peptic                  10
Name: Disease, Length: 149, dtype: int64


In [0]:
symptom_data = symptom_data.sort_values(by=['Disease'], ignore_index=True)

In [0]:
symptom_data = pd.concat([symptom_data]*20, ignore_index=True)

In [0]:
symptom_data = symptom_data.to_csv("/content/drive/My Drive/gtdt/symptoms_data.csv", index=False)
symptom_data = pd.read_csv("/content/drive/My Drive/gtdt/symptoms_data.csv")

In [36]:
symptom_data

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29795,upper respiratory infection,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
29796,upper respiratory infection,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
29797,upper respiratory infection,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
29798,upper respiratory infection,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [0]:
x = symptom_data[cols]
y = symptom_data['Disease']

In [0]:
#Splitting data into train test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

**Training the new created dataset on different classifiers**

In [40]:
print ("DecisionTree")
dt = DecisionTreeClassifier(max_depth=120)
clf_dt=dt.fit(x_train,y_train)
print ("Acurracy: ", clf_dt.score(x_test,y_test))

DecisionTree
Acurracy:  0.8824486475493187


In [41]:
mnb = MultinomialNB()
clf_mnb=mnb.fit(x_train,y_train)
print ("Acurracy: ", clf_mnb.score(x_test,y_test))

Acurracy:  0.8798047589993899


In [42]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=12)
clf_rfc=rfc.fit(x_train,y_train)
print ("Acurracy: ", clf_rfc.score(x_test,y_test))

Acurracy:  0.8224527150701647


In [43]:
naivebys = GaussianNB()
clf_naivebys=naivebys.fit(x_train,y_train)
print ("Acurracy: ", clf_naivebys.score(x_test,y_test))

Acurracy:  0.8834655277608298


In [44]:
svm = svm.SVC()
clf_svm=svm.fit(x_train,y_train)
print ("Acurracy: ", clf_svm.score(x_test,y_test))

Acurracy:  0.8849908480780964


**Feature selection for disease prediction**

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]

In [0]:
features = cols

In [0]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

Taking symptoms as input features

In [48]:
input_features = [feature_dict['withdraw'], feature_dict['agitation'], feature_dict['hemiplegia']]  # These symptoms are of parkinson's disease
input_features

[401, 15, 159]

In [0]:
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)

In [0]:
arr = np.array(arr).reshape(-1,len(arr))

**Predicted disease with all models**

In [53]:
dt.predict(arr)

array(['delirium'], dtype=object)

In [54]:
mnb.predict(arr)

array(['parkinson disease'], dtype='<U36')

In [55]:
rfc.predict(arr)

array(['HIV'], dtype=object)

In [56]:
naivebys.predict(arr)

array(['parkinson disease'], dtype='<U36')

In [58]:
svm.predict(arr)

array(['parkinson disease'], dtype=object)

We can see that Multinomial NB, NB and SVM predict the correct disease.